In [1]:
# Main import block
import random
import pandas as pd
import requests
import json
from bs4 import BeautifulSoup
import numpy as np
from datetime import date
import datetime
import multiprocessing

import time
from tqdm import tqdm

# Upload to google sheets
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials

import signal
from contextlib import contextmanager

import warnings
warnings.filterwarnings('ignore')


----

In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import json

# Установка пути к драйверу Firefox (Gecko)
driver_path = '../selenium/geckodriver'

'''# Создание экземпляра профиля Firefox
profile = webdriver.FirefoxProfile()
profile.set_preference("general.useragent.override", "Mozilla/115.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")
profile.set_preference("dom.webdriver.enabled", False)
profile.set_preference("useAutomationExtension", False)
profile.update_preferences()'''

# Создание экземпляра драйвера Firefox
driver = webdriver.Firefox(executable_path=driver_path)#, firefox_profile=profile)

# Переход на страницу
url = 'https://www.lme.com/en/Metals/Non-ferrous/LME-Copper#Trading+day+summary'
driver.get(url)

# Нахождение элемента с JSON данными на странице с помощью селектора CSS
element = driver.find_element_by_css_selector('script[type="application/json"]')

# Получение содержимого элемента
json_data = element.get_attribute('innerHTML')

# Закрытие браузера
driver.quit()

# Обработка полученного JSON
data = json.loads(json_data)

# Вывод полученных данных
print(data)

____

In [35]:
url = 'https://www.westmetall.com/en/markdaten.php?action=table&field=LME_Al_cash'
response = requests.get(url)

page = BeautifulSoup(response.text, 'html.parser')
page_elements = page.find_all('td')

aluminium_date = pd.to_datetime(page_elements[0].text)
aluminium_settlment = page_elements[1].text

data_row = pd.DataFrame(data={'date':aluminium_date, 'aluminium': aluminium_settlment}, index=[0])
data_row

,date,aluminium
0,2023-07-31,"2,195.50"


---

In [25]:
def westmetall():
    def aluminium():
        url = 'https://www.westmetall.com/en/markdaten.php?action=table&field=LME_Al_cash'
        response = requests.get(url)

        page = BeautifulSoup(response.text, 'html.parser')
        page_elements = page.find_all('td')

        aluminium_date = pd.to_datetime(page_elements[0].text)
        aluminium_settlment = page_elements[1].text

        aluminium = pd.DataFrame(
            data={'date': aluminium_date, 'aluminium': aluminium_settlment}, index=[0])
        #aluminium['aluminium'] = pd.to_numeric(aluminium['aluminium'])
        
        return aluminium

    def copper():
        url = 'https://www.westmetall.com/en/markdaten.php?action=table&field=LME_Cu_cash'
        response = requests.get(url)

        page = BeautifulSoup(response.text, 'html.parser')
        page_elements = page.find_all('td')

        copper_date = pd.to_datetime(page_elements[0].text)
        copper_settlment = page_elements[1].text

        copper = pd.DataFrame(
            data={'date': copper_date, 'copper': copper_settlment}, index=[0])
        #copper['copper'] = pd.to_numeric(copper['copper'])
        
        return copper

    def lead():
        url = 'https://www.westmetall.com/en/markdaten.php?action=table&field=LME_Pb_cash'
        response = requests.get(url)

        page = BeautifulSoup(response.text, 'html.parser')
        page_elements = page.find_all('td')

        lead_date = pd.to_datetime(page_elements[0].text)
        lead_settlment = page_elements[1].text

        lead = pd.DataFrame(
            data={'date': lead_date, 'lead': lead_settlment}, index=[0])
        #lead['lead'] = pd.to_numeric(lead['lead'])
        
        return lead

    def nickel():
        url = 'https://www.westmetall.com/en/markdaten.php?action=table&field=LME_Ni_cash'
        response = requests.get(url)

        page = BeautifulSoup(response.text, 'html.parser')
        page_elements = page.find_all('td')

        nickel_date = pd.to_datetime(page_elements[0].text)
        nickel_settlment = page_elements[1].text

        nickel = pd.DataFrame(
            data={'date': nickel_date, 'nickel': nickel_settlment}, index=[0])
        #nickel['nickel'] = pd.to_numeric(nickel['nickel'])
        
        return nickel

    def zink():
        url = 'https://www.westmetall.com/en/markdaten.php?action=table&field=LME_Zn_cash'
        response = requests.get(url)

        page = BeautifulSoup(response.text, 'html.parser')
        page_elements = page.find_all('td')

        zink_date = pd.to_datetime(page_elements[0].text)
        zink_settlment = page_elements[1].text

        zink = pd.DataFrame(
            data={'date': zink_date, 'zink': zink_settlment}, index=[0])
        #zink['zink'] = pd.to_numeric(zink['zink'])
        
        return zink

    al_data = aluminium()
    cu_data = copper()
    ld_data = lead()
    nk_data = nickel()
    zk_data = zink()

    result = pd.merge(pd.merge(pd.merge(pd.merge(al_data, cu_data, on='date', suffixes=['_al', '_cu']),
                                        ld_data, on='date'), nk_data, on='date'), zk_data, on='date')
    
    result['aluminium'] = result['aluminium'].str.replace(',', '')
    result['copper'] = result['copper'].str.replace(',', '')
    result['lead'] = result['lead'].str.replace(',', '')
    result['nickel'] = result['nickel'].str.replace(',', '')
    result['zink'] = result['zink'].str.replace(',', '')
    
    df = pd.read_excel('./data/LME_westmetall_db.xlsx', index_col=0)
    df = pd.concat([df, result], axis=0)
    df = df.drop_duplicates()
    df.reset_index(drop=True)
    
    google_upload(df, 'LME_westmetall')

    with pd.ExcelWriter(
        "../parser_beta/data/LME_westmetall_db.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD") as writer:
        df.to_excel(writer, sheet_name='LME_westmetall')

    return df


In [26]:
row = westmetall()

In [27]:
row

,date,aluminium,copper,lead,nickel,zink
0,2023-07-31,2195.50,8635.50,2150.00,21625.00,2550.00


In [22]:
row['aluminium'].str.replace(',', '')

0    2195.50
Name: aluminium, dtype: object

---

In [29]:
df = pd.read_excel('./data/LME_westmetall_db.xlsx', index_col=0)
df = pd.concat([df, row], axis=0)
df.reset_index(drop=True)

,date,aluminium,copper,lead,nickel,zink
0,2022-01-04,2815.5,9660.0,2327.0,20730,3602.0
1,2022-01-05,2866.0,9778.0,2343.0,20900,3660.0
2,2022-01-06,2912.5,9565.0,2291.0,20480,3590.0
3,2022-01-07,2919.5,9615.0,2342.0,20725,3602.5
4,2022-01-10,2923.0,9665.0,2303.0,21045,3576.5
...,...,...,...,...,...,...
391,2023-07-25,2184.0,8597.0,2157.0,21350,2447.0
392,2023-07-26,2170.5,8534.0,2140.0,21575,2455.0
393,2023-07-27,2176.0,8578.5,2170.0,21395,2485.0
394,2023-07-28,2161.5,8592.0,2157.0,21650,2485.0


In [ ]:
df.